<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [1]:
#Spatiotemporal range and wavelengths/band of interest are defined

#Temporal range is defined
start_of_epoch = '2016-12-01'
end_of_epoch =  '2017-01-30'

#Wavelengths/bands of interest are defined
bands_of_interest = ['green',
                     'red', 
                     'nir',
                     'swir1']

#Sensors of interest are defined
sensors = ['ls8']#,"
    #'ls7',
    #'ls5' ] 


#Create bounding box around the location of the stream gauge
lat_max = -19.79
lat_min = -19.85
lon_max = 145.34
lon_min = 145.27

#Create query
query = {'time': (start_of_epoch, end_of_epoch)}
query['x'] = (lon_min, lon_max)
query['y'] = (lat_max, lat_min)
query['crs'] = 'EPSG:4326'

In [2]:
#import csv
import os
import sys
#import datetime

In [3]:
#get the DEA version of the plotting functions
sys.path.append(os.path.abspath('/g/data/r78/rjd547/jupyter_notebooks/dea-notebooks/algorithms/'))
import DEADataHandling

In [4]:
## DEADataHandling.py
'''
This file contains a set of python functions for handling data within DEA.
Available functions:
load_nbarx

Last modified: March 2018
Author: Claire Krause

'''

from datacube.helpers import ga_pq_fuser
from datacube.storage import masking
from datacube import Datacube
dc = Datacube(app = 'test')

In [14]:
def load_nbarx(sensor, query, bands_of_interest, product = 'nbart'): 
    '''loads nbar or nbart data for a sensor, masks using pq, then filters 
    out terrain -999s
    Last modified: March 2018
    Author: Bex Dunn
    Modified by: Claire Krause
    
    This function requires the following be loaded:
    from datacube.helpers import ga_pq_fuser
    from datacube.storage import masking
    from datacube import Datacube
    dc = Datacube(app = 'test')
    
    inputs
    sensor - Options are 'ls5', 'ls7', 'ls8'
    query - A dict containing the query bounds. Can include lat/lon, time etc
    bands_of_interest - List of strings containing the bands to be read in. Options
                       'red', 'green', 'blue', 'nir', 'swir1', 'swir2'

    optional
    product - 'nbar' or 'nbart'. Defaults to nbart unless otherwise specified

    outputs
    ds - Extracted and pq filtered dataset
    crs - ds coordinate reference system
    affine - ds affine
    '''  
    dataset = []
    product_name = '{}_{}_albers'.format(sensor, product)
    print('loading {}'.format(product_name))
    ds = dc.load(product = product_name, measurements = bands_of_interest,
                 group_by = 'solar_day', **query)
    #grab crs defs from loaded ds if ds exists
    if ds.variables:
        crs = ds.crs
        affine = ds.affine
        print('loaded {}'.format(product_name))
        mask_product = '{}_{}_albers'.format(sensor, 'pq')
        sensor_pq = dc.load(product = mask_product, fuse_func = ga_pq_fuser,
                            group_by = 'solar_day', **query)
        if sensor_pq.variables:
            print('making mask {}'.format(mask_product))
            cloud_free = masking.make_mask(sensor_pq.pixelquality,
                                           cloud_acca ='no_cloud',
                                           cloud_shadow_acca = 'no_cloud_shadow',                           
                                           cloud_shadow_fmask = 'no_cloud_shadow',
                                           cloud_fmask = 'no_cloud',
                                           blue_saturated = False,
                                           green_saturated = False,
                                           red_saturated = False,
                                           nir_saturated = False,
                                           swir1_saturated = False,
                                           swir2_saturated = False,
                                           contiguous = True)
            ds = ds.where(cloud_free)
            ds.attrs['crs'] = crs
            ds.attrs['affine'] = affine
            
        if product=='nbart':
            print('masked {} with {} and filtered terrain'.format(product_name,
                                                                  mask_product))
            # nbarT is correctly used to correct terrain by replacing -999.0 with nan
            ds = ds.where(ds!=-999.0)
        elif product=='nbar':
            print('masked {} with {}'.format(product_name, mask_product))
        else: 
            print('did not mask {} with {}'.format(product_name, mask_product))
    else:
        print ('did not load {}'.format(product_name)) 

    if len(ds.variables) > 0:
        return ds, crs, affine
    else:
        return None

In [ ]:
data, crs, affine = DEADataHandling.load_nbarx('ls8', query, bands_of_interest, product='nbar')

loading ls8_nbar_albers


In [13]:
if product=='nbart':
            print('masked {} with {} and filtered terrain')#.format(product_name,
                                                  #                mask_product))
elif product=='nbar':
            print('masked {} with {}')#.format(product_name, mask_product))

masked {} with {} and filtered terrain


In [12]:
product='nbart'

In [15]:
data, crs, affine = DEADataHandling.load_nbarx('ls8', query, bands_of_interest, product='nbar')

loading ls8_nbar_albers
loaded ls8_nbar_albers
making mask ls8_pq_albers
masked ls8_nbar_albers with ls8_pq_albers
